In [39]:
import os
import gc
import sys
srcpath = os.path.realpath('../SourceFiles')
sys.path.append(srcpath)
#from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

In [40]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [41]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.0.101::5025::SOCKET'

inst = TEVisaInst(inst_addr)

resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp)

connected to: Tabor Electronics,P25812B,000000220726,1.118.0


In [42]:
# Set the default paranoia level. 
# It controls the default behavior of: inst.send_scpi_cmd(scpi_str).
#  - paranoia-level = 0: send scpi_str as is.
#  - paranoia-level = 1: append '*OPC?' to scpi_str, and send it as query 
#  - paranoia-level = 2: append ':SYST:ERR?' to scpi_str, send it as query, and print warnning in case of error.

inst.default_paranoia_level = 2 # paranoia-level = 2 (good for debugging)

# Reset the instrument
inst.send_scpi_cmd('*CLS; *RST')
resp = inst.send_scpi_query(":SYST:ERR?")
print(resp)

0, no error


In [43]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get model dependant parameters:

if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    
half_dac = max_dac / 2.0

# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = (int(resp) // 64) * 64
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_seglen = arbmem_capacity // bpp
print ('Max segment length: {0:,}'.format(max_seglen))



Model: P25812B 
Number of channels: 12
Max segment number: 65536
Available memory per DDR: 4,286,578,688 wave-bytes
Max segment length: 2,143,289,344


In [44]:
seglen=202400000
num_cycles=2000
cyclelen = seglen / num_cycles

#sin_wav=pyte.build_sine_wave(cycle_len,num_cycles)
x = np.linspace(start=0, stop=2 * np.pi * num_cycles, num=seglen, endpoint=False)
y = (np.sin(x) + 1.0) * half_dac
del x

y = np.round(y)
y = np.clip(y, 0, max_dac)

wave = y.astype(wpt_type)
del y



In [45]:
# data-type code: 'B' = uint8, 'H' = uint16
#data_type = 'H' if wpt_type == np.uint16 else 'B'


channb=1
segnum=1


cmd = ':INST:CHAN {0}'.format(channb)
inst.send_scpi_cmd(cmd)
    
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
inst.send_scpi_cmd(cmd)

cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

# Write the data in chunks of 64M points
# This may take time please wait...

chunk = 64 * 1024 * 1024
offset = 0

#start_time = time.time()
tic = time.perf_counter()

while offset < seglen:
    
    chunk = min(chunk, seglen - offset)
    
    cmd = ':TRAC:DATA {0}'.format(offset * offset_factor)
    inst.write_binary_data(cmd, wave[offset:offset + chunk])
    
    offset = offset + chunk

#inst.write_binary_values('*OPC?; :TRAC:DATA', wave, datatype=data_type)
#end_time = time.time()
#time_lapsed = end_time - start_time

#print('Downloaded the wave in "{0}" seconds'.format(time_lapsed))

toc = time.perf_counter()
timer = float(toc-tic)

print('Downloaded the wave in "{0}" seconds\n'.format(timer))

#resp = inst.read();
#inst.timeout = 10000;

# Play the specified segment at the selected channel:
cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

# Turn on the output of the selected channel:
inst.send_scpi_cmd(':OUTP ON')
    
resp = inst.send_scpi_query(':SYST:ERR?')
print('System error: ' + resp)

  

Downloaded the wave in "24.326041799999985" seconds

System error: 0, no error


In [30]:
del wave

# close connection
inst.close_instrument()